In [ ]:
import pandas as pd
import numpy as np
import requests 
import re
import os
from bs4 import BeautifulSoup

In [ ]:
# Job categories list
categories = ['SERVICE_INDUSTRY', 'INFORMATION_TECHNOLOGY', 'TRADE', 'STATE_PUBLIC_ADMIN', 'EDUCATION_SCIENCE', 'SALES', 'ORGANISATION_MANAGEMENT', 'PRODUCTION_MANUFACTURING', 'ADMINISTRATION',
'LOGISTICS_TRANSPORT', 'FINANCE_ACCOUNTING', 'TOURISM_HOTELS_CATERING', 'TECHNICAL_ENGINEERING', 'CONSTRUCTION_REAL_ESTATE', 'HEALTH_SOCIAL_CARE', 'BANKING_INSURANCE', 'ELECTRONICS_TELECOM',
'ENERGETICS_ELECTRICITY', 'MARKETING_ADVERTISING', 'SECURITY_RESCUE_DEFENCE', 'LAW_LEGAL', 'OTHER', 'CULTURE_ENTERTAINMENT', 'MEDIA_PR', 'HUMAN_RESOURCES', 'QUALITY_ASSURANCE',
'AGRICULTURE_ENVIRONMENTAL', 'FOREST_WOODCUTTING', 'INTERNSHIP', 'THIRD_SECTOR_NGO', 'PHARMACY', 'SEASONAL']

In [ ]:
# Create a list of first pages for each category
first_pages_list = []
for i in categories:
    first_pages_list.append(f"https://cv.ee/en/search?limit=20&offset=0&categories%5B0%5D={i}")
first_pages_list 

In [ ]:
# Find total number of jobs and create a list of pages' URLs
all_pages_list = []

for url in first_pages_list:

    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, features='html')

    total_jobs = soup.find('div', class_ = 'search-results-heading').text
    total_jobs_number =  int((re.findall(r'\d+', total_jobs))[0])

    url_values_range = np.arange(0, total_jobs_number, 20)

    for i in url_values_range:
        all_pages_list.append(f"https://www.cv.ee/en/search?limit=20&offset={i}&categories%5B0%5D={url.rsplit('=', 1)[1]}")

all_pages_list

In [ ]:
# Create a list of jobs' basic information from all pages
jobs_list = []

for page in all_pages_list:
    response = requests.get(page)
    html = response.text
    soup = BeautifulSoup(html, features='html')
    jobs = soup.find('ul', class_ = 'vacancies-list')

    for job in jobs:
        title = job.find(class_ = 'vacancy-item__title').text
        location = job.find(class_ = 'vacancy-item__locations').text
        company = job.find(class_ = 'vacancy-item__body').find('a', class_ = "jsx-145194818").text

        ### Find url
        url = job.find_all('a', class_ = 'jsx-145194818 vacancy-item')
        url_list = []
        for i in url:
            url_list.append('https://www.cv.ee' + i.attrs['href'])
    
        jobs_list.append({
            'title': title,
            'location': location,
            'company': company,
            'url': url_list[0],
            'category': page.rsplit('=', 1)[1]
        }) 

In [ ]:
# As a job can be in multiple categories the number of records is higher than the actual total job offers
print(len(jobs_list))

In [ ]:
# Create jobs dataframe
df = pd.DataFrame(jobs_list)
df

In [ ]:
# Add job IDs from the URLs
id_list = []

for i in df['url']:
    id_list.append((str(i)).split('/')[5])

df['id'] = id_list
df['id'] = df['id'].astype(int)

In [ ]:
# Group df by category and put categories to a list
df = df.groupby(['id', 'title', 'company', 'location', 'url'])['category'].apply(list).reset_index()
df

In [ ]:
# Dataframe len should be equal to the actual number of job offers
print(len(df))

In [ ]:
# Create a list of URLs to get additional information
url_list = df['url'].values.tolist()

In [ ]:
# Get additional information about job vacancies (deadline, work type, language and salary)   
work_deadline = []
work_type = []
work_lang= []
work_salary = []

for url in url_list:

    response = requests.get(url)
    html = response.text
    soup = BeautifulSoup(html, features='html')

    # Deadline
    deadlines = soup.find_all(class_= 'jsx-4256297253 vacancy-info__deadline')
    deadline_list = []
    try:
        for i in deadlines:
            deadline_list.append((i.text.replace(u'\xa0', u'')))
        work_deadline.append(deadline_list[0]) 
    except IndexError:
        work_deadline.append(None)

    # Language
    languages = soup.find_all('li', class_= 'jsx-930987492')
    lang_list = []
    for i in languages:
        lang_list.append(i.text.replace(u'\xa0', u''))

    work_lang.append(lang_list)

    # Work type                            
    type_list = []
    types = soup.find_all('ul', class_= 'jsx-930987492 vacancy-highlights__section-list')
    try:
        if len(types) == 2 and len(lang_list) > 0:
            for i in types[0]:
                type_list.append(i.text.replace(u'\xa0', u''))
        else:
            for i in types[1]:
                type_list.append(i.text.replace(u'\xa0', u'')) 
    except IndexError: 
        type_list.append(None)

    work_type.append(type_list)

    # Salary
    salary = soup.find_all(class_= 'jsx-930987492 vacancy-highlights__salary-amount')
    salaries_list = []
    try:
        for i in salary:
            salaries_list.append(((i.get_text(strip=True, separator='\n').splitlines()))[-1])
        work_salary.append(salaries_list[0])
    except IndexError: 
        work_salary.append(None)

In [ ]:
# Append values to the dataframe
df['language'] = work_lang
df['work_type'] = work_type
df['deadline'] = work_deadline
df['salary'] = work_salary

In [ ]:
# Split location into three columns for further analysis
df['location_rev'] = df['location'].str.replace(',', '')
df['location_rev'] = df['location_rev'].str.split().apply(reversed).apply(' '.join)
df[['country', 'region', 'city']] = df['location_rev'].str.split(expand=True)
df.drop(columns=['location_rev'], inplace=True)

In [ ]:
# Convert deadline to datetime format
df['deadline'] = ((df['deadline'].str.split(':').str[-1]).str.slice(0, 11))
df['deadline'] = pd.to_datetime(df['deadline'], dayfirst=True)

In [ ]:
# Get first number of the salary and convert to numeric
df['salary'].str.split('-', expand=True)[0]
df['salary']= pd.to_numeric(df['salary'], errors='coerce')

In [ ]:
# Get categories dummies as a separate df
category = df['category']
df_cat = pd.get_dummies(category.explode()).groupby(level=0).sum()
df_categories = df[['id']].join(df_cat)
df_categories.columns = df_categories.columns.str.lower()
df_categories.columns = df_categories.columns.str.title()
df_categories = df_categories.rename(columns=lambda name: name.replace('_', ' ')) 
df_categories     

In [ ]:
# Get language dummies as a separate df
language = df['language']
df_lang = pd.get_dummies(language.explode()).groupby(level=0).sum()
df_languages = df[['id']].join(df_lang)
df_languages

In [ ]:
# Get work type dummies as a separate df
worktype = df['work_type']
df_worktype = pd.get_dummies(worktype.explode()).groupby(level=0).sum()
df_worktype = df[['id']].join(df_worktype)
df_worktype

In [ ]:
# Drop unnecessary columns from the main dataframe
df.drop(columns=['category', 'language', 'work_type'], inplace=True)

In [ ]:
# Export dataframes for further visualisation
df.to_excel(os.path.join('output', 'jobs_main.xlsx'))
df_categories.to_excel(os.path.join('output', 'jobs_categories.xlsx'))
df_languages.to_excel(os.path.join('output', 'jobs_languages.xlsx'))
df_worktype.to_excel(os.path.join('output', 'jobs_worktype.xlsx'))